<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/relax_amber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#relax your structure (using amber)

In [ ]:
#@title install amber
from IPython.utils import io
import os
import subprocess
import tqdm.notebook
from sys import version_info 
python_version = f"{version_info.major}.{version_info.minor}"

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

if not os.path.isfile("stereo_chemical_props.txt"):
  try:
    with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
      with io.capture_output() as captured:
        # Install py3dmol.
        %shell pip install py3dmol
        pbar.update(4)

        # Install OpenMM and pdbfixer.
        %shell rm -rf /opt/conda
        %shell wget -q -P /tmp \
          https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
            && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
            && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
        pbar.update(15)

        PATH=%env PATH
        %env PATH=/opt/conda/bin:{PATH}
        %shell conda install -qy conda==4.13.0 \
            && conda install -qy -c conda-forge \
              python={python_version} \
              openmm=7.5.1 \
              pdbfixer
        pbar.update(80)

        %shell wget -q -P /content \
          https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
        pbar.update(1)
  except subprocess.CalledProcessError:
    print(captured)
    raise

In [ ]:
#@title install AlphaFold
import sys
if not os.path.isdir("/content/alphafold"):
  GIT_REPO = 'https://github.com/deepmind/alphafold'
  try:
    with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
      with io.capture_output() as captured:
        %shell rm -rf alphafold
        %shell git clone --branch main {GIT_REPO} alphafold
        pbar.update(10)
        # Install the required versions of all dependencies.
        %shell pip3 install -r ./alphafold/requirements.txt
        # Run setup.py to install only AlphaFold.
        %shell pip3 install --no-dependencies ./alphafold
        pbar.update(90)

        # Apply OpenMM patch.
        %shell pushd /opt/conda/lib/python{python_version}/site-packages/ && \
            patch -p0 < /content/alphafold/docker/openmm.patch && \
            popd

        # Make sure stereo_chemical_props.txt is in all locations where it could be searched for.
        %shell mkdir -p /content/alphafold/alphafold/common
        %shell cp -f /content/stereo_chemical_props.txt /content/alphafold/alphafold/common
        %shell mkdir -p /opt/conda/lib/python{python_version}/site-packages/alphafold/common/
        %shell cp -f /content/stereo_chemical_props.txt /opt/conda/lib/python{python_version}/site-packages/alphafold/common/

  except subprocess.CalledProcessError:
    print(captured)
    raise

if "/content/alphafold" not in sys.path:
  # Make sure everything we need is on the path.
  sys.path.append(f"/opt/conda/lib/python{python_version}/site-packages")
  sys.path.append('/content/alphafold')

import jax
if jax.local_devices()[0].platform == 'cpu':
  DEVICE = "CPU"
else:
  DEVICE = "GPU"
print(f'Will run on {DEVICE}')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from alphafold.relax import relax
from alphafold.relax import utils
from alphafold.common import protein, residue_constants

MODRES = {'MSE':'MET','MLY':'LYS','FME':'MET','HYP':'PRO',
          'TPO':'THR','CSO':'CYS','SEP':'SER','M3L':'LYS',
          'HSK':'HIS','SAC':'SER','PCA':'GLU','DAL':'ALA',
          'CME':'CYS','CSD':'CYS','OCS':'CYS','DPR':'PRO',
          'B3K':'LYS','ALY':'LYS','YCM':'CYS','MLZ':'LYS',
          '4BF':'TYR','KCX':'LYS','B3E':'GLU','B3D':'ASP',
          'HZP':'PRO','CSX':'CYS','BAL':'ALA','HIC':'HIS',
          'DBZ':'ALA','DCY':'CYS','DVA':'VAL','NLE':'LEU',
          'SMC':'CYS','AGM':'ARG','B3A':'ALA','DAS':'ASP',
          'DLY':'LYS','DSN':'SER','DTH':'THR','GL3':'GLY',
          'HY3':'PRO','LLP':'LYS','MGN':'GLN','MHS':'HIS',
          'TRQ':'TRP','B3Y':'TYR','PHI':'PHE','PTR':'TYR',
          'TYS':'TYR','IAS':'ASP','GPL':'LYS','KYN':'TRP',
          'CSD':'CYS','SEC':'CYS'}

def pdb_to_string(pdb_file, chains=None, models=[1]):
  '''read pdb file and return as string'''

  if chains is not None:
    if "," in chains: chains = chains.split(",")
    if not isinstance(chains,list): chains = [chains]
  if models is not None:
    if not isinstance(models,list): models = [models]

  modres = {**MODRES}
  lines = []
  seen = []
  model = 1
  for line in open(pdb_file,"rb"):
    line = line.decode("utf-8","ignore").rstrip()
    if line[:5] == "MODEL":
      model = int(line[5:])
    if models is None or model in models:
      if line[:6] == "MODRES":
        k = line[12:15]
        v = line[24:27]
        if k not in modres and v in residue_constants.restype_3to1:
          modres[k] = v
      if line[:6] == "HETATM":
        k = line[17:20]
        if k in modres:
          line = "ATOM  "+line[6:17]+modres[k]+line[20:]
      if line[:4] == "ATOM":
        chain = line[21:22]
        if chains is None or chain in chains:
          atom = line[12:12+4].strip()
          resi = line[17:17+3]
          resn = line[22:22+5].strip()
          if resn[-1].isalpha(): # alternative atom
            resn = resn[:-1]
            line = line[:26]+" "+line[27:]
          key = f"{model}_{chain}_{resn}_{resi}_{atom}"
          if key not in seen: # skip alternative placements
            lines.append(line)
            seen.append(key)
      if line[:5] == "MODEL" or line[:3] == "TER" or line[:6] == "ENDMDL":
        lines.append(line)
  return "\n".join(lines)

def relax_me(pdb_in, pdb_out):
  pdb_str = pdb_to_string(pdb_in)
  protein_obj = protein.from_pdb_string(pdb_str)
  amber_relaxer = relax.AmberRelaxation(
    max_iterations=0,
    tolerance=2.39,
    stiffness=10.0,
    exclude_residues=[],
    max_outer_iterations=3,
    use_gpu=DEVICE == "GPU")
  relaxed_pdb_lines, _, _ = amber_relaxer.process(prot=protein_obj)
  with open(pdb_out, 'w') as f:
      f.write(relaxed_pdb_lines)

In [ ]:
#@title #RELAX
#@markdown - Click the little ▶ play icon to the left to get an upload prompt.
#@markdown - After relax is done, `relaxed.pdb` will automatically download.
#@markdown - If download was blocked, click the little folder 📁 icon on the left, right-click `relaxed.pdb` and select Download!

from google.colab import files
pdb_dict = files.upload()
relax_me(pdb_in=list(pdb_dict.keys())[0],
         pdb_out="relaxed.pdb")
files.download(f'relaxed.pdb')